# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing]. 
Please download the data and place it in your data_path at "./data."

Installing needed packages

In [85]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

This is so that you do not have to repeat ./data/ for every single call to the data

In [86]:
data_path = './data/'

This loads in the data, normalizes the data then converts them to PyTorch tensors

In [87]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

train_data_mean = np.mean(train_data)
train_data_std = np.std(train_data)
train_data_normalized = (train_data - train_data_mean) / train_data_std
test_data_normalized = (test_data - train_data_mean) / train_data_std

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data_normalized)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data_normalized)
y_test_tensor = torch.LongTensor(test_label)

In [88]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [89]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Build simple Deep learning model

This creates the model

In [90]:
class EEGCNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGCNNClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(64, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(negative_slope=0.05),
            nn.Dropout(0.05),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(32, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.LeakyReLU(negative_slope=0.05),
            nn.Dropout(0.05),
            nn.MaxPool1d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 198, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

This sets all the parameters needed for the training.

In [91]:
num_classes = 5
model = EEGCNNClassifier(num_classes).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


This will train the model. Changing the number of epochs and other hyper parameters will change the amount of time that it needs to run and changes the accuracy.

In [92]:
num_epochs = 50
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        data = data.view(-1, 64, 795)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 1.736389398574829
Epoch 2/50, Loss: 1.6980760097503662
Epoch 3/50, Loss: 1.687290906906128
Epoch 4/50, Loss: 1.6932255029678345
Epoch 5/50, Loss: 1.5915905237197876
Epoch 6/50, Loss: 1.5352710485458374
Epoch 7/50, Loss: 1.4041122198104858
Epoch 8/50, Loss: 1.3590201139450073
Epoch 9/50, Loss: 1.2975069284439087
Epoch 10/50, Loss: 1.2665941715240479
Epoch 11/50, Loss: 1.259460210800171
Epoch 12/50, Loss: 1.2797644138336182
Epoch 13/50, Loss: 1.1810084581375122
Epoch 14/50, Loss: 1.0131717920303345
Epoch 15/50, Loss: 1.2328523397445679
Epoch 16/50, Loss: 1.3824858665466309
Epoch 17/50, Loss: 0.9771748781204224
Epoch 18/50, Loss: 1.0219978094100952
Epoch 19/50, Loss: 1.1935224533081055
Epoch 20/50, Loss: 1.1556110382080078
Epoch 21/50, Loss: 1.1515525579452515
Epoch 22/50, Loss: 1.0399763584136963
Epoch 23/50, Loss: 0.8630117177963257
Epoch 24/50, Loss: 0.939810037612915
Epoch 25/50, Loss: 1.0592700242996216
Epoch 26/50, Loss: 1.0252130031585693
Epoch 27/50, Loss: 1.1464

Evaluates the model and gives a testing accuracy

In [93]:
model.eval() # Evaluate your model
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 39.63%
